In [30]:
# import librosa
import sys
sys.path.append('/Users/angelinazhai/ffmpeg') #https://stackoverflow.com/questions/57350259/filenotfounderror-errno-2-no-such-file-or-directory-ffprobe-ffprobe

# import scipy
from scipy import signal
import matplotlib.pyplot as plt
import os
import numpy as np

import librosa
import librosa.display

import pickle
import pandas as pd

import scipy


In [31]:
#relocate to folder with recordings
ROOT = '/Users/angelinazhai/Desktop/Github-Repositories/language-dsp'
ORIGINAL_REC = 'recordings/original'

In [25]:
#processing every single file in the folder

processed_info = {}

def update_dict():
    for root, dirs, files in os.walk(os.path.join(ROOT, ORIGINAL_REC)):
        for file in files:
            if file.endswith('.m4a'):
                y, sr = librosa.load(os.path.join(ROOT, ORIGINAL_REC, file))
                processed_info[file] = {}
                processed_info[file]['SR'] = sr
                processed_info[file]['Time_Domain'] = y
                D = librosa.stft(y)
                processed_info[file]['Freq_Domain'] = D
                S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
                processed_info[file]['db'] = S_db


#pickle the processed info into new pickle file
def to_pickle_file():
    with open('processed_info.pickle', 'wb') as handle:
        pickle.dump(processed_info, handle, protocol=pickle.HIGHEST_PROTOCOL)

# print(processed_info)


In [12]:
plt.figure()
librosa.display.specshow(S_db)
plt.colorbar()
plt.show()
processed_info['Ree Eng.m4a']['Freq_Domain']

array([[-6.9987855e-06+0.00000000e+00j, -5.7704787e-04+0.00000000e+00j,
         4.5725766e-01+0.00000000e+00j, ...,
         3.6071774e-02+0.00000000e+00j, -2.3769058e-02+0.00000000e+00j,
         4.8439503e-02+0.00000000e+00j],
       [-6.5216868e-06-2.46768354e-06j, -1.3076197e-04-4.91960905e-04j,
         3.9145032e-01+2.33251780e-01j, ...,
        -1.2828460e-02-1.24639392e-01j,  6.9628984e-02+6.79009855e-02j,
        -6.5461732e-02+8.48506205e-03j],
       [-5.1659486e-06-4.56822227e-06j,  2.9855379e-04-1.55156406e-04j,
         2.0453095e-01+3.91696304e-01j, ...,
         5.4374378e-02+1.81370810e-01j, -9.0492452e-03-1.25641108e-01j,
         4.4900995e-02+1.67616480e-03j],
       ...,
       [ 2.4546820e-10+2.40369304e-11j,  6.4327689e-11+1.67369585e-09j,
        -1.8037914e-08+6.46509264e-08j, ...,
        -1.8901669e-07+6.31924824e-09j, -4.5563991e-07+8.96415543e-07j,
         8.1764729e-06-1.17970585e-05j],
       [ 2.3627414e-10+1.07686872e-11j, -1.0340614e-09+5.07039966e-1

In [18]:
# #uncomment for double checking spectrogram

# plt.figure()
# librosa.display.specshow(processed_info['Anne Canto.m4a']['db'])
# plt.colorbar()
# plt.show()

In [27]:
#test opening pickle file
def dict_from_pickle():
    with open('processed_info.pickle', 'rb') as handle:
        b = pickle.load(handle)
    return b
    

In [56]:
# conduct t test on frequency information of two recordings
def t_test_freq(info, r1, r2):
    r1 = info[r1]['Freq_Domain']
    r2 = info[r2]['Freq_Domain']

    #flatten arrays
    r1 = r1.flatten()
    r2 = r2.flatten()

    t, p = scipy.stats.ttest_ind(r1, r2)
    return t, p




In [59]:
#read in dict from pickle file
info = dict_from_pickle()

#conduct t test on two recordings
t1, p1 = t_test_freq(info, 'Ree Eng.m4a', 'Ree French.m4a')
print(abs(t1))

t2, p2 = t_test_freq(info, 'Ang English.m4a', 'Ang Shanghai.m4a')
print(abs(t2))

t3, p3 = t_test_freq(info, 'Nat English.m4a', 'Nat Indonesian .m4a') #there is a space
print(abs(t3))

t4, p4 = t_test_freq(info, 'Anne Eng.m4a', 'Anne Canto.m4a')
print(abs(t4))

0.039417788713595824


0.03469226472525642
0.0456987345100065
0.03816357871405698
